Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
pickle_file = 'Data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

## Logistic regression

In [23]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Adding L2-Regularization.
    c = 0.005
    l2 = tf.nn.l2_loss(weights)

    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + c * l2
    )

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [16]:
num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(
                predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 32.407951
Training accuracy: 8.7%
Validation accuracy: 10.9%
Loss at step 100: 10.541798
Training accuracy: 73.5%
Validation accuracy: 71.1%
Loss at step 200: 6.267365
Training accuracy: 77.2%
Validation accuracy: 74.1%
Loss at step 300: 3.849798
Training accuracy: 79.7%
Validation accuracy: 75.8%
Loss at step 400: 2.471223
Training accuracy: 81.6%
Validation accuracy: 77.5%
Loss at step 500: 1.685480
Training accuracy: 83.2%
Validation accuracy: 79.0%
Loss at step 600: 1.236290
Training accuracy: 84.2%
Validation accuracy: 80.1%
Loss at step 700: 0.978157
Training accuracy: 84.8%
Validation accuracy: 80.7%
Loss at step 800: 0.829000
Training accuracy: 85.1%
Validation accuracy: 81.1%
Test accuracy: 88.1%


## Neural network part

In [18]:
batch_size = 128
node_size = 1024

graph = tf.Graph()

def init_weights(shape):
    weights = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(weights)

def init_biases(shape):
    biases = tf.constant(0.1, shape=shape)
    return tf.Variable(biases)

with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables of layer 1.
    weights_1 = init_weights([image_size*image_size, node_size])
    biases_1 = init_biases([node_size])
    layer_1_output = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
    
    # Variables of hidden(2) layer.
    weights_2 = init_weights([node_size, node_size])
    biases_2 = init_biases([node_size])
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights_2) + biases_2)

    # Training computation.
    weights_output = init_weights([node_size, num_labels])
    biases_output = init_biases([num_labels])
    # Adding L2-Regularization.
    c = 0.0005
    l2 = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(weights_output) + \
         tf.nn.l2_loss(biases_1) + tf.nn.l2_loss(biases_2) + tf.nn.l2_loss(biases_output)
        
    logits = tf.matmul(layer_2_output, weights_output) + biases_output
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + c * l2
    )

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    # Valid & Test.
    valid_prediction_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
    valid_prediction_2 = tf.nn.relu(tf.matmul(valid_prediction_1, weights_2) + biases_2)
    valid_logit = tf.matmul(valid_prediction_2, weights_output) + biases_output
    valid_prediction = tf.nn.softmax(valid_logit)
    
    test_prediction_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
    test_prediction_2 = tf.nn.relu(tf.matmul(test_prediction_1, weights_2) + biases_2)
    test_logit = tf.matmul(test_prediction_2, weights_output) + biases_output
    test_prediction = tf.nn.softmax(test_logit)

In [19]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 10.217555
Minibatch accuracy: 8.6%
Validation accuracy: 18.7%
Minibatch loss at step 500: 3.846623
Minibatch accuracy: 90.6%
Validation accuracy: 84.5%
Minibatch loss at step 1000: 3.395413
Minibatch accuracy: 78.1%
Validation accuracy: 86.5%
Minibatch loss at step 1500: 2.563380
Minibatch accuracy: 85.2%
Validation accuracy: 87.4%
Minibatch loss at step 2000: 1.846843
Minibatch accuracy: 93.8%
Validation accuracy: 87.7%
Minibatch loss at step 2500: 1.636855
Minibatch accuracy: 93.0%
Validation accuracy: 88.1%
Minibatch loss at step 3000: 1.483486
Minibatch accuracy: 85.9%
Validation accuracy: 88.1%
Test accuracy: 94.1%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [43]:
reduced_dataset = train_dataset[: 1000, :]
reduced_labels = train_labels[: 1000, :]

## Logistic regression overfitting

In [37]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(reduced_dataset[:train_subset, :])
    tf_train_labels = tf.constant(reduced_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Adding L2-Regularization.
    c = 0.005
    l2 = tf.nn.l2_loss(weights)

    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + c * l2
    )

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)


num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(
                predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 37.080307
Training accuracy: 10.0%
Validation accuracy: 12.3%
Loss at step 100: 9.138417
Training accuracy: 100.0%
Validation accuracy: 19.5%
Loss at step 200: 5.543088
Training accuracy: 100.0%
Validation accuracy: 20.8%
Loss at step 300: 3.364175
Training accuracy: 100.0%
Validation accuracy: 23.0%
Loss at step 400: 2.043741
Training accuracy: 100.0%
Validation accuracy: 25.1%
Loss at step 500: 1.243649
Training accuracy: 100.0%
Validation accuracy: 27.4%
Loss at step 600: 0.758923
Training accuracy: 100.0%
Validation accuracy: 28.1%
Loss at step 700: 0.465278
Training accuracy: 100.0%
Validation accuracy: 29.8%
Loss at step 800: 0.287387
Training accuracy: 100.0%
Validation accuracy: 30.8%
Test accuracy: 33.6%


## Neural network overfitting

In [52]:
batch_size = 128
node_size = 1024

graph = tf.Graph()

def init_weights(shape):
    weights = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(weights)

def init_biases(shape):
    biases = tf.constant(0.1, shape=shape)
    return tf.Variable(biases)

with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables of layer 1.
    weights_1 = init_weights([image_size*image_size, node_size])
    biases_1 = init_biases([node_size])
    layer_1_output = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
    
    # Variables of hidden(2) layer.
    weights_2 = init_weights([node_size, node_size])
    biases_2 = init_biases([node_size])
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights_2) + biases_2)

    # Training computation.
    weights_output = init_weights([node_size, num_labels])
    biases_output = init_biases([num_labels])
   
    logits = tf.matmul(layer_2_output, weights_output) + biases_output
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
    )

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    # Valid & Test.
    valid_prediction_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
    valid_prediction_2 = tf.nn.relu(tf.matmul(valid_prediction_1, weights_2) + biases_2)
    valid_logit = tf.matmul(valid_prediction_2, weights_output) + biases_output
    valid_prediction = tf.nn.softmax(valid_logit)
    
    test_prediction_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
    test_prediction_2 = tf.nn.relu(tf.matmul(test_prediction_1, weights_2) + biases_2)
    test_logit = tf.matmul(test_prediction_2, weights_output) + biases_output
    test_prediction = tf.nn.softmax(test_logit)
    
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step % 5 * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = reduced_dataset[offset:(offset + batch_size), :]
        batch_labels = reduced_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 6.883548
Minibatch accuracy: 8.6%
Validation accuracy: 27.1%
Minibatch loss at step 500: 0.022443
Minibatch accuracy: 99.2%
Validation accuracy: 74.8%
Minibatch loss at step 1000: 0.036637
Minibatch accuracy: 99.2%
Validation accuracy: 74.0%
Minibatch loss at step 1500: 0.020367
Minibatch accuracy: 99.2%
Validation accuracy: 75.7%
Minibatch loss at step 2000: 0.037749
Minibatch accuracy: 98.4%
Validation accuracy: 75.5%
Minibatch loss at step 2500: 0.038204
Minibatch accuracy: 98.4%
Validation accuracy: 75.5%
Minibatch loss at step 3000: 0.026134
Minibatch accuracy: 98.4%
Validation accuracy: 75.7%
Test accuracy: 83.1%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [61]:
batch_size = 128
node_size = 1024

graph = tf.Graph()

def init_weights(shape):
    weights = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(weights)

def init_biases(shape):
    biases = tf.constant(0.1, shape=shape)
    return tf.Variable(biases)

with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables of layer 1.
    weights_1 = init_weights([image_size*image_size, node_size])
    biases_1 = init_biases([node_size])
    layer_1_output = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
    keep_prop = tf.placeholder(tf.float32)
    dropout_1 = tf.nn.dropout(layer_1_output, keep_prop)
    
    # Variables of hidden(2) layer.
    weights_2 = init_weights([node_size, node_size])
    biases_2 = init_biases([node_size])
    layer_2_output = tf.nn.relu(tf.matmul(dropout_1, weights_2) + biases_2)
    dropout_2 = tf.nn.dropout(layer_2_output, keep_prop)
    
    # Training computation.
    weights_output = init_weights([node_size, num_labels])
    biases_output = init_biases([num_labels])
    # Adding L2-Regularization.
    c = 0.0005
    l2 = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(weights_output) + \
         tf.nn.l2_loss(biases_1) + tf.nn.l2_loss(biases_2) + tf.nn.l2_loss(biases_output)
        
    logits = tf.matmul(dropout_2, weights_output) + biases_output
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + c * l2
    )

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.3).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    # Valid & Test.
    valid_prediction_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
    valid_prediction_2 = tf.nn.relu(tf.matmul(valid_prediction_1, weights_2) + biases_2)
    valid_logit = tf.matmul(valid_prediction_2, weights_output) + biases_output
    valid_prediction = tf.nn.softmax(valid_logit)
    
    test_prediction_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
    test_prediction_2 = tf.nn.relu(tf.matmul(test_prediction_1, weights_2) + biases_2)
    test_logit = tf.matmul(test_prediction_2, weights_output) + biases_output
    test_prediction = tf.nn.softmax(test_logit)
    
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prop: 0.5}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 20.801682
Minibatch accuracy: 13.3%
Validation accuracy: 28.0%
Minibatch loss at step 500: 3.713473
Minibatch accuracy: 85.2%
Validation accuracy: 82.7%
Minibatch loss at step 1000: 3.533412
Minibatch accuracy: 75.8%
Validation accuracy: 83.9%
Minibatch loss at step 1500: 2.880638
Minibatch accuracy: 83.6%
Validation accuracy: 84.6%
Minibatch loss at step 2000: 2.318574
Minibatch accuracy: 89.1%
Validation accuracy: 85.4%
Minibatch loss at step 2500: 2.262842
Minibatch accuracy: 83.6%
Validation accuracy: 85.8%
Minibatch loss at step 3000: 2.103068
Minibatch accuracy: 82.0%
Validation accuracy: 85.9%
Test accuracy: 92.5%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [11]:
batch_size = 128
node_size = 1024

graph = tf.Graph()

def init_weights(shape):
    weights = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(weights)

def init_biases(shape):
    biases = tf.constant(0.1, shape=shape)
    return tf.Variable(biases)

with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables of layer 1.
    weights_1 = init_weights([image_size*image_size, node_size])
    biases_1 = init_biases([node_size])
    layer_1_output = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
    keep_prop = tf.placeholder(tf.float32)
    dropout_1 = tf.nn.dropout(layer_1_output, keep_prop)
    
    # Variables of hidden(2) layer.
    weights_2 = init_weights([node_size, node_size])
    biases_2 = init_biases([node_size])
    layer_2_output = tf.nn.relu(tf.matmul(dropout_1, weights_2) + biases_2)
    dropout_2 = tf.nn.dropout(layer_2_output, keep_prop)
    
    # Training computation.
    weights_output = init_weights([node_size, num_labels])
    biases_output = init_biases([num_labels])
    # Adding L2-Regularization.
    c = 0.0005
    l2 = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(weights_output) + \
         tf.nn.l2_loss(biases_1) + tf.nn.l2_loss(biases_2) + tf.nn.l2_loss(biases_output)
        
    logits = tf.matmul(dropout_2, weights_output) + biases_output
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + c * l2
    )

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.0\3).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    # Valid & Test.
    valid_prediction_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
    valid_prediction_2 = tf.nn.relu(tf.matmul(valid_prediction_1, weights_2) + biases_2)
    valid_logit = tf.matmul(valid_prediction_2, weights_output) + biases_output
    valid_prediction = tf.nn.softmax(valid_logit)
    
    test_prediction_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
    test_prediction_2 = tf.nn.relu(tf.matmul(test_prediction_1, weights_2) + biases_2)
    test_logit = tf.matmul(test_prediction_2, weights_output) + biases_output
    test_prediction = tf.nn.softmax(test_logit)
    
num_steps = 4001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prop: 0.75}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 14.368759
Minibatch accuracy: 11.7%
Validation accuracy: 28.5%
Minibatch loss at step 500: 4.416716
Minibatch accuracy: 82.0%
Validation accuracy: 82.8%
Minibatch loss at step 1000: 4.279666
Minibatch accuracy: 80.5%
Validation accuracy: 84.3%
Minibatch loss at step 1500: 4.250067
Minibatch accuracy: 78.1%
Validation accuracy: 85.2%
Minibatch loss at step 2000: 3.712409
Minibatch accuracy: 89.1%
Validation accuracy: 85.6%
Minibatch loss at step 2500: 3.834310
Minibatch accuracy: 84.4%
Validation accuracy: 85.8%
Minibatch loss at step 3000: 3.912828
Minibatch accuracy: 81.2%
Validation accuracy: 86.1%
Minibatch loss at step 3500: 3.737841
Minibatch accuracy: 85.2%
Validation accuracy: 86.4%
Minibatch loss at step 4000: 3.568515
Minibatch accuracy: 87.5%
Validation accuracy: 86.6%
Test accuracy: 93.0%
